In [2]:
#Creating a workspace using from_config()
from azureml.core import Workspace

ws = Workspace.from_config()

In [5]:
#Creating a workspace using .get()
from azureml.core import Workspace

ws = Workspace.get(name='ignite_machinelearn',
                   subscription_id='2d00455f-76c3-4759-ade7-8798d7d0c0da',
                   resource_group='az900-rg1')

In [6]:
#Retreiving the workspaces created and their details
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print(compute.name, ":", compute.type)

ignite-instance1 : ComputeInstance
ignite-instance2 : AmlCompute


In [7]:
#Creating an experiment in general
from azureml.core import Experiment

# create an experiment variable
experiment = Experiment(workspace = ws, name = "my-experiment")

# start the experiment
run = experiment.start_logging()

# experiment code goes here

# end the experiment
run.complete()

In [11]:
#Creating an experiment and loading the dataset in that
from azureml.core import Experiment
import pandas as pd

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = 'my-experiment')

# Start logging data from the experiment
run = experiment.start_logging()

# load the dataset and count the rows
data = pd.read_csv('diabetes.csv')
row_count = (len(data))

# Log the row count
run.log('observations', row_count)

# Complete the experiment
run.complete()

In [12]:
#Retreiving the details of experiments run
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [14]:
#Retreiving the results/metrics of experiments run
import json

# Get logged metrics
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

{
  "observations": 15000
}


In [22]:
#Retreiving the location of output file
import json

files = run.get_file_names()
print(json.dumps(files, indent=2))

[
  "outputs/sample.csv"
]


In [21]:
#Can place the output file as per user needs
run.upload_file(name='outputs/sample.csv', path_or_stream='./sample.csv')

In [23]:
#Creating and running the experiment using Run
from azureml.core import Run
import pandas as pd
import matplotlib.pyplot as plt
import os

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
data = pd.read_csv('diabetes.csv')

# Count the rows and log the result
row_count = (len(data))
run.log('observations', row_count)

# Save a sample of the data
os.makedirs('outputs', exist_ok=True)
data.sample(100).to_csv("outputs/sample.csv", index=False, header=True)

# Complete the run
run.complete()

Attempted to log scalar metric observations:
15000


In [48]:
#Training the machine learning model on Azure ML SDK Python
from azureml.core import Run
import argparse
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Get the experiment run context
run = Run.get_context()



# Prepare the dataset
data = pd.read_csv('diabetes.csv')
X, y = data[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, data['Diabetic'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# Train a logistic regression model
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# Save the trained model
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='model.pkl')

run.complete()

Attempted to log scalar metric Accuracy:
0.7831111111111111


In [49]:
#Registering the model to save it to workspace
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='classification_model',
                       model_path='model.pkl', # local path
                       description='A classification model',
                       tags={'data-format': 'CSV'},
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.20.3')

Registering model classification_model


In [50]:
#Retreiving the saved model and its details
from azureml.core import Model

for model in Model.list(ws):
    # Get model name and auto-generated version
    print(model.name, 'version:', model.version)

classification_model version: 1
amlstudio-predict-diabetes version: 1


In [55]:
#Retreiving the datastores created in our workspace and its details
for ds_name in ws.datastores:
    print(ds_name)

blob_data
azureml_globaldatasets
workspaceworkingdirectory
workspaceartifactstore
workspacefilestore
workspaceblobstore


In [59]:
#Creating the blob store
blob_store = Datastore.get(ws, datastore_name='blob_data')
blob_store

{
  "name": "blob_data",
  "container_name": "data-container",
  "account_name": "az900strgeacc",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [60]:
#Retreiving the default data store
default_store = ws.get_default_datastore()
default_store

{
  "name": "blob_data",
  "container_name": "data-container",
  "account_name": "az900strgeacc",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [61]:
#Setting the default data store
ws.set_default_datastore('blob_data')
blob_store

{
  "name": "blob_data",
  "container_name": "data-container",
  "account_name": "az900strgeacc",
  "protocol": "https",
  "endpoint": "core.windows.net"
}